In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from langchain.chat_models import ChatOpenAI

from langchain.callbacks import StreamingStdOutCallbackHandler

# set_llm_cache
# LM의 응답 저장
# 같은 질문을 받을 때 답변을 생성하지 않고, 이미 답변한 답을 캐싱을 이용해 답변
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache()) # 모든 respons가 메모리에 저장

chat = ChatOpenAI(
    temperature=0.1,
)


In [11]:
# 캐싱에 시간이 필요함.
# 13초 가량 걸림

chat.predict("How do you make italian pasta")

"To make Italian pasta, follow these steps:\n\n1. Start by boiling a large pot of water. Add salt to the water to enhance the flavor of the pasta.\n\n2. Once the water is boiling, add the pasta to the pot. The amount of pasta depends on how many servings you want to make. Generally, 2 ounces (56 grams) of dried pasta per person is a good estimate.\n\n3. Cook the pasta according to the package instructions. Different types of pasta have different cooking times, so make sure to check the package for the recommended cooking time. Stir occasionally to prevent the pasta from sticking together.\n\n4. While the pasta is cooking, prepare the sauce. Italian pasta can be served with various sauces such as tomato sauce, pesto, carbonara, or Alfredo sauce. Choose your preferred sauce and follow a recipe or prepare it according to your taste.\n\n5. Once the pasta is cooked al dente (cooked but still firm to the bite), drain it in a colander. Reserve a small amount of the pasta water if you want to 

In [14]:
chat.predict("How do you make italian pasta")

"To make Italian pasta, follow these steps:\n\n1. Start by boiling a large pot of water. Add salt to the water to enhance the flavor of the pasta.\n\n2. Once the water is boiling, add the pasta to the pot. The amount of pasta depends on how many servings you want to make. Generally, 2 ounces (56 grams) of dried pasta per person is a good estimate.\n\n3. Cook the pasta according to the package instructions. Different types of pasta have different cooking times, so make sure to check the package for the recommended cooking time. Stir occasionally to prevent the pasta from sticking together.\n\n4. While the pasta is cooking, prepare the sauce. Italian pasta can be served with various sauces such as tomato sauce, pesto, carbonara, or Alfredo sauce. Choose your preferred sauce and follow a recipe or prepare it according to your taste.\n\n5. Once the pasta is cooked al dente (cooked but still firm to the bite), drain it in a colander. Reserve a small amount of the pasta water if you want to 